# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:

from azureml.core import Workspace, Experiment

ws = Workspace.get(name="capstone-ml")
exp = Experiment(workspace=ws, name="uda-cap")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
Workspace name: capstone-ml
Azure region: eastus
Subscription id: b09642af-8e07-4efe-80f5-7f7e59fb2cf2
Resource group: uda-cap


In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone-ml'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute


from azureml.core.compute_target import ComputeTargetException

cluster_name = "cap-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('I found the existing cluster, So I am using it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           vm_priority = 'lowpriority',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

I found the existing cluster, So I am using it.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=1,
    delay_evaluation=5
)
#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "C": uniform(0.0, 1.0), 
    "max_iter": choice(50, 100, 150, 200, 250)
})

#TODO: Create your estimator and hyperdrive config compute_target=compute_target,
estimator = SKLearn(
    source_directory=".",
    compute_target=compute_target,
    entry_script="train.py"
)

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=40
)

In [5]:
#TODO: Submit your experiment

hyperdrive_run = exp.submit(hyperdrive_run_config, show_output=True)

# Run Details
# OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

# TODO: In the cell below, use the RunDetails widget to show the different experiments.

RunDetails(hyperdrive_run).show()



WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
uda-cap,HD_158fc58f-0811-44d9-b6d7-58f87d9e21a8,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
import joblib
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']



print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:', parameter_values[1])
print('\n max_iter:', parameter_values[3])



Best Run Id:  HD_158fc58f-0811-44d9-b6d7-58f87d9e21a8_5

 Accuracy: 0.8666666666666667

 C: 0.8831212105861365

 max_iter: 250


In [9]:
#TODO: Save the best model

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])


# save model
best_run.download_file("/outputs/model.joblib", "linreg_best_model.joblib")

Best Run Id:  HD_158fc58f-0811-44d9-b6d7-58f87d9e21a8_5

 Accuracy: 0.8666666666666667


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [24]:
from azureml.core.model import Model
from azureml.core.resource_configuration import ResourceConfiguration
model = Model.register(workspace=ws,
                          model_name='my-model',
                          model_path='./linreg_best_model.joblib')
                          

Registering model my-model


### Environment Details

In [43]:
from azureml.core import Environment

curated_env = Environment.get(workspace=ws, name="AzureML-Scikit-learn-0.20.3")

In [52]:

from azureml.core.environment import CondaDependencies
conda_dep = CondaDependencies()

# conda_dep.add_pip_package("joblib")
conda_dep.add_conda_package("joblib")
curated_env.python.conda_dependencies=conda_dep

In [53]:
print("packages", curated_env.python.conda_dependencies.serialize_to_string())

packages # Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- joblib
channels:
- anaconda
- conda-forge



In [58]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


inference_config = InferenceConfig(entry_script='./score.py',
                                    environment=curated_env)


In [59]:
Model.deploy(ws, 'modelcapstone5', [model], inference_config)

AciWebservice(workspace=Workspace.create(name='capstone-ml', subscription_id='b09642af-8e07-4efe-80f5-7f7e59fb2cf2', resource_group='uda-cap'), name=modelcapstone5, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={'azureml.git.repository_uri': 'https://github.com/bleso-a/nd00333-capstone.git', 'mlflow.source.git.repoURL': 'https://github.com/bleso-a/nd00333-capstone.git', 'azureml.git.branch': 'master', 'mlflow.source.git.branch': 'master', 'azureml.git.commit': '7800ab0b20649343c596fc11f9a01a2bc7ce3595', 'mlflow.source.git.commit': '7800ab0b20649343c596fc11f9a01a2bc7ce3595', 'azureml.git.dirty': 'True'})

TODO: In the cell below, send a request to the web service you deployed to test it.

In [60]:
import requests
import json


In [62]:
scoring_uri = 'http://278a33ab-b3b1-45a7-ac83-64e1b5d9ec0a.eastus.azurecontainer.io/score'

In [63]:
data = {'data':
[
    '{"Married":1.0,"Dependents":0.0,"Education":1.0,"Self_Employed":1.0,"ApplicantIncome":3000.0,"CoapplicantIncome":0.0,"LoanAmount":66.0,"Loan_Amount_Term":360.0,"Credit_History":1.0,"Property_Area":2.0,"Loan_Status":1.0}'
]
}

### Endpoint Result

In [66]:


# Set the content type
headers = {'Content-Type': 'application/json'}



# Make the request and display the response
resp = requests.post(scoring_uri, data, headers=headers)
print(resp.json())


{"result": 1.0 }
